In [32]:
import os
import glob
import numpy as np
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
import gradio as gr
import time
from sklearn.manifold import TSNE
import plotly.graph_objects as go

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_ollama import ChatOllama

# What will we do

- Part A: We will divide our documents into CHUNKS
- Part B: We will encode our CHUNKS into VECTORS and put in Chroma
- Part C: We will visualize our vectors

## 1. Devide our documents into CHUNKS

In [39]:
# First load all our knowledge-base folder
folders = glob.glob("knowledge-base/**/*")
print(f"Found {len(folders)} files in the knowledge base") # 17 files: 

## How many characters in all the documents?
entire_knowledge_base = ""

for file_path in folders:
    with open(file_path, 'r', encoding='utf-8') as f:
        entire_knowledge_base += f.read()
        entire_knowledge_base += "\n\n"

print(f"Total characters in knowledge base: {len(entire_knowledge_base):,}") # 101,404 words

Found 17 files in the knowledge base
Total characters in knowledge base: 101,404


In [40]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

#text_loader_kwargs = {'encoding': 'utf-8'}
# Nếu dòng trên không hoạt động, người dùng Windows có thể dùng dòng dưới thay thế
text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print("Total documents loaded:", len(documents))

Total documents loaded: 17


In [7]:
documents[0]

Document(metadata={'source': 'knowledge-base/visas/D-4 General Trainee Visa.md', 'doc_type': 'visas'}, page_content='# Visa Học Tập D-4 (D-4 General Trainee Visa)\n\n## Tổng Quan\nVisa D-4 là loại visa dành cho các hoạt động học tập, nghiên cứu và đào tạo ngắn hạn tại Hàn Quốc, bao gồm học tiếng Hàn, các khóa đào tạo nghề và chương trình trao đổi sinh viên.\n\n## Phân Loại Visa D-4\n\n### D-4-1: Học Tiếng Hàn\n- Khóa học tiếng Hàn tại các trường đại học\n- Thời gian: 3 tháng - 2 năm\n\n### D-4-2: Nghiên Cứu\n- Chương trình nghiên cứu ngắn hạn\n- Thực tập nghiên cứu\n\n### D-4-6: Đào Tạo Nghề\n- Các khóa đào tạo kỹ năng nghề nghiệp\n- Chương trình đào tạo kỹ thuật\n\n### D-4-7: Chương Trình Không Cấp Bằng\n- Chương trình trao đổi sinh viên\n- Khóa học ngắn hạn\n\n## Đối Tượng Áp Dụng\n- Sinh viên muốn học tiếng Hàn\n- Người tham gia chương trình đào tạo nghề\n- Sinh viên trao đổi quốc tế\n- Người tham gia các khóa đào tạo ngắn hạn\n\n## Thời Hạn Visa\n- **Thời gian cấp:** 6 tháng - 2 nă

In [41]:
# Devide into CHUNKS
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Slightly smaller chunks for better retrieval
    chunk_overlap=200,  # Reduced overlap for performance
    separators=["\n\n", "\n", ". ", " ", ""]  # Better separation
)

chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")
print(f"First chunk:\n\n{chunks[0]}")

Created 128 chunks
First chunk:

page_content='# Visa Học Tập D-4 (D-4 General Trainee Visa)

## Tổng Quan
Visa D-4 là loại visa dành cho các hoạt động học tập, nghiên cứu và đào tạo ngắn hạn tại Hàn Quốc, bao gồm học tiếng Hàn, các khóa đào tạo nghề và chương trình trao đổi sinh viên.

## Phân Loại Visa D-4

### D-4-1: Học Tiếng Hàn
- Khóa học tiếng Hàn tại các trường đại học
- Thời gian: 3 tháng - 2 năm

### D-4-2: Nghiên Cứu
- Chương trình nghiên cứu ngắn hạn
- Thực tập nghiên cứu

### D-4-6: Đào Tạo Nghề
- Các khóa đào tạo kỹ năng nghề nghiệp
- Chương trình đào tạo kỹ thuật

### D-4-7: Chương Trình Không Cấp Bằng
- Chương trình trao đổi sinh viên
- Khóa học ngắn hạn

## Đối Tượng Áp Dụng
- Sinh viên muốn học tiếng Hàn
- Người tham gia chương trình đào tạo nghề
- Sinh viên trao đổi quốc tế
- Người tham gia các khóa đào tạo ngắn hạn

## Thời Hạn Visa
- **Thời gian cấp:** 6 tháng - 2 năm
- **Gia hạn:** Có thể gia hạn tối đa 2 năm
- **Thời gian xử lý:** 3-7 ngày làm việc

## Yêu Cầu Cơ

## 2. Encode CHUNKS into vectors and store in Chroma

we already setup and save hugging face into .env file now time to use that

In [7]:
load_dotenv(override=True)

True

In [42]:
# choose an embedding model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [43]:
# Đặt tên cho database vector (có thể tùy chọn)
db_name = "vector_db"

# Kiểm tra nếu database Chroma đã tồn tại, thì xóa collection để khởi động lại từ đầu or remove
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()



In [44]:
# Tạo vector store bằng Chroma
vectorstore = Chroma.from_documents(
    documents=chunks,              # Danh sách các đoạn văn bản đã chia nhỏ
    embedding=embeddings,          # Hàm embedding (HuggingFace)
    persist_directory=db_name      # Thư mục lưu trữ database
)
# Kiểm tra số lượng document đã được lưu vào vector store
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 128 documents


In [45]:
# #Lấy ra bộ sưu tập vector từ vectorstore
collection = vectorstore._collection

In [46]:
# ------investiage our vectors-----------------
# #Lấy 1 embedding từ database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

# #Kiểm tra số chiều (số phần tử trong vector)
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")
# -------------------------------------------------

The vectors have 384 dimensions


In [ ]:
# sample_embedding # look like this

## 3. Visualization

In [47]:
# Prepare
result = collection.get(include=['embeddings', 'documents', 'metadatas']) # Lấy toàn bộ vector, tài liệu và metadata từ collection
vectors = np.array(result['embeddings']) # Đưa embedding vào mảng numpy
documents = result['documents'] # Lưu lại văn bản
doc_types = [metadata['doc_type'] for metadata in result['metadatas']] # Trích loại tài liệu từ metadata (giả sử có 'doc_type')
colors = [['blue', 'green', 'red', 'orange'][['company', 'employees', 'visas', 'schools'].index(t)] for t in doc_types] # Gán màu sắc tùy theo loại tài liệu

In [48]:
# Giảm số chiều của vector xuống 3D bằng t-SNE (T-distributed Stochastic Neighbor Embedding)

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Tạo biểu đồ scatter 2D
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Loại: {t}<br>Văn bản: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='Biểu đồ 3D Chroma Vector Store',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

# Chatbot nôị bộ - Công ty KorStudy!

In [49]:
retriever = vectorstore.as_retriever()

MODEL = "llama3.2"
llm = ChatOllama(temperature=0.7, model=MODEL)

In [50]:
retriever.invoke("Who is Lan?")

[Document(id='49154c24-d794-4241-8328-ebee1673eef1', metadata={'doc_type': 'employees', 'source': 'knowledge-base/employees/Alex Kim.md'}, page_content='## Lịch Làm Việc\n\n### Thứ 2-6\n- **8:00-12:00**: Tư vấn trực tiếp tại văn phòng\n- **13:00-17:00**: Xử lý hồ sơ và liên hệ trường đối tác\n- **17:00-18:00**: Tư vấn online cho học sinh tỉnh xa\n\n### Thứ 7-CN\n- **Thứ 7**: Tư vấn và workshop cho phụ huynh\n- **Chủ nhật**: Tư vấn online và emergency support\n\n## Feedback Từ Học Sinh\n\n> *"Anh Alex rất tận tâm và hiểu rõ tâm lý học sinh. Nhờ anh mà em đã vào được trường mơ ước là Korea University. Anh luôn động viên em trong những lúc khó khăn nhất."*  \n> **- Nguyễn Minh Hương, Korea University Business School 2023**\n\n> *"Alex đã giúp em từ việc chọn trường phù hợp đến chuẩn bị hồ sơ một cách chi tiết. Tỷ lệ thành công của anh ấy là có lý do cả!"*  \n> **- Trần Đức Nam, Seoul National University Engineering 2022**\n\n---\n\n*Để đặt lịch tư vấn với Alex, vui lòng liên hệ qua hotlin

In [51]:
llm.invoke("Who is Lan?")

AIMessage(content='There are several individuals and characters named "Lan" across different fields, so it\'s possible that you\'re referring to one of the following:\n\n1. Lan Lampe: A Danish professional footballer who plays as a midfielder for the Denmark women\'s national team.\n2. Lan Laryon: A South Korean rapper and singer under the stage name "Lan".\n3. Lan Lan: A Chinese actress, singer, and television personality.\n\nWithout more context or information, it\'s difficult to determine which "Lan" you\'re referring to. If you could provide more details or clarify who Lan is in your context, I\'d be happy to try and provide a more specific answer.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-12-26T13:55:01.308485Z', 'done': True, 'done_reason': 'stop', 'total_duration': 27061007712, 'load_duration': 6883404432, 'prompt_eval_count': 29, 'prompt_eval_duration': 3184219505, 'eval_count': 139, 'eval_duration': 16976131599, 'logprobs': None, 'mode

In [52]:
SYSTEM_PROMPT_TEMPLATE = """
Bạn là chuyên gia tư vấn du học Hàn Quốc tại trung tâm Korea Study. 
Nhiệm vụ của bạn là trả lời các câu hỏi liên quan đến trung tâm, nhân viên, trường học và thông tin visa một cách ngắn gọn và chính xác. 
Nếu có thông tin liên quan trong ngữ cảnh được cung cấp, hãy sử dụng để trả lời câu hỏi.
Nếu bạn không biết câu trả lời, hãy nói rõ rằng bạn không biết. Tuyệt đối không bịa thông tin nếu không có ngữ cảnh phù hợp.
Ngữ cảnh:
{context}
"""

In [53]:
def answer_question(question: str, history):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
    response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    return response.content

In [54]:
answer_question("Who is Lan", [])

'Lan là bà Nguyễn Thị Lan, người đang giữ chức vụ Giám Đốc Điều hành (CEO) của trung tâm Korea Study tại Việt Nam. Cô là một chuyên gia tư vấn du học Hàn Quốc có kinh nghiệm và trình độ cao, với bằng cấp từ Đại học Yonsei ở Seoul, Hàn Quốc, và chứng chỉ từ các tổ chức quốc tế như ICEF và PMP.\n\nBà Lan đã có hơn 15 năm kinh nghiệm trong lĩnh vực tư vấn du học và giáo dục quốc tế, và đã giúp hàng ngàn học sinh Việt Nam đạt được mục tiêu du học mơ ước của mình. Cô cũng là một người có kiến thức sâu về hệ thống giáo dục Hàn Quốc, đặc biệt là các trường đại học danh giá như Korea University, Seoul National University, và những thông tin cập nhật từ các tổ chức này.\n\nBà Lan cũng có khả năng giao tiếp tốt với nhiều ngôn ngữ, bao gồm Tiếng Việt (bản ngữ), Tiếng Hàn (Thành thạo - TOPIK 6), và Tiếng Anh (Thành thạo - IELTS 8.0).'

# Yayy!!!!! But ... wait! ---> What should we do next ? 🤔

In [55]:
# Test questions file 
from test_questions import save_tests_to_jsonl, print_test_summary

save_tests_to_jsonl()
print_test_summary()

Saved 40 test questions to tests.jsonl

=== Test Question Summary ===
Total: 40 questions

company_info: 8 questions
employee_info: 7 questions
school_info: 10 questions
visa_info: 8 questions
complex: 7 questions


In [ ]:
# Test Single Question (Quick Check)
from test_questions import load_tests_from_jsonl
from evaluate_rag import evaluate_retrieval, evaluate_answer_with_llm

# Load tests
tests = load_tests_from_jsonl()

# Pick one test
test = tests[0]
print(f"Question: {test.question}")
print(f"Expected keywords: {test.keywords}")

# Evaluate retrieval
retrieval_metrics, chunks = evaluate_retrieval(
    question=test.question,
    expected_keywords=test.keywords,
    retriever=retriever,  # Your retriever from setup
    k=10
)

print(f"\nRetrieval Results:")
print(f"  MRR: {retrieval_metrics.mrr:.3f}")
print(f"  nDCG: {retrieval_metrics.ndcg:.3f}")
print(f"  Keywords found: {retrieval_metrics.keywords_found}/{retrieval_metrics.total_keywords}")
print(f"  Coverage: {retrieval_metrics.keyword_coverage:.1f}%")

# Generate answer
generated_answer = answer_question(test.question, [])
print(f"\nGenerated Answer:\n{generated_answer}")

# Evaluate answer (using LLM judge)
answer_metrics = evaluate_answer_with_llm(
    question=test.question,
    generated_answer=generated_answer,
    reference_answer=test.reference_answer,
    llm=llm  
)

print(f"\nAnswer Evaluation:")
print(f"  Accuracy: {answer_metrics.accuracy:.2f}/5")
print(f"  Completeness: {answer_metrics.completeness:.2f}/5")
print(f"  Relevance: {answer_metrics.relevance:.2f}/5")
print(f"  Feedback: {answer_metrics.feedback}")

Question: Korea Study được thành lập năm nào?
Expected keywords: ['2018', 'thành lập', 'năm']

Retrieval Results:
  MRR: 1.000
  nDCG: 0.973
  Keywords found: 3/3
  Coverage: 100.0%

Generated Answer:
Korea Study được thành lập vào năm 2018.

Answer Evaluation:
  Accuracy: 5.00/5
  Completeness: 4.00/5
  Relevance: 5.00/5
  Feedback: The generated answer is factually correct, but it lacks some details from the reference answer. It provides a brief overview of Korea Study's establishment year and its mission, which is relevant to answering the question.


In [57]:
from evaluate_rag import run_full_evaluation, analyze_by_category

# Run full evaluation (automated with LLM judge)
results = run_full_evaluation(
    retriever=retriever,           # Your retriever
    answer_function=answer_question,  # Your answer function
    use_manual_eval=False,         # Use LLM for evaluation
    llm=llm,                       # Your LLM
    test_file="tests.jsonl"
)

# Analyze by category
analyze_by_category(results)


Running evaluation on 40 test questions


Test 1/40: company_info
Question: Korea Study được thành lập năm nào?
  MRR: 1.000
  nDCG: 0.973
  Coverage: 100.0%
  Accuracy: 5.00/5
  Completeness: 4.00/5
  Relevance: 5.00/5

Test 2/40: company_info
Question: Văn phòng Korea Study ở đâu?
  MRR: 0.100
  nDCG: 0.130
  Coverage: 20.0%


KeyboardInterrupt: 